In [44]:

# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from pprint import pprint
api_format = "json"
from sqlalchemy import create_engine

Part 1- GETTING DATA

In [2]:
# for loop for different API 
li=[]
name_q=['games','teams','tips']

for q in name_q:
    s="https://api.squiggle.com.au/?q="+q+";format=json"
    li.append(s)

In [3]:
li[0]

'https://api.squiggle.com.au/?q=games;format=json'

In [4]:
#p print to check the json data
#pprint(requests.get(li[0]).json())

In [5]:
#convering json data into dataframe 
df_games=pd.DataFrame.from_dict(pd.json_normalize(requests.get(li[0]).json()['games']),orient='columns')

In [6]:
df_games.head()

,id,ascore,is_final,venue,complete,updated,is_grand_final,ateam,hteamid,date,...,tz,hscore,ateamid,hteam,winnerteamid,round,localtime,agoals,hbehinds,year
0,1,132,0,M.C.G.,100,2017-04-15 15:59:16,0,Richmond,3,2017-03-23 19:20:00,...,+11:00,89,14,Carlton,14.0,1,2017-03-23 19:20:00,20,5,2017
1,2,100,0,M.C.G.,100,2017-04-15 15:59:16,0,Western Bulldogs,4,2017-03-24 19:50:00,...,+11:00,86,18,Collingwood,18.0,1,2017-03-24 19:50:00,15,14,2017
2,3,110,0,S.C.G.,100,2017-04-15 15:59:16,0,Port Adelaide,16,2017-03-25 16:35:00,...,+11:00,82,13,Sydney,13.0,1,2017-03-25 16:35:00,17,10,2017
3,4,120,0,Docklands,100,2017-04-15 15:59:16,0,Melbourne,15,2017-03-25 16:35:00,...,+11:00,90,11,St Kilda,11.0,1,2017-03-25 16:35:00,18,12,2017
4,5,98,0,Carrara,100,2017-04-15 15:59:16,0,Brisbane Lions,8,2017-03-25 20:05:00,...,+11:00,96,2,Gold Coast,2.0,1,2017-03-25 19:05:00,15,12,2017


In [47]:
#clean data-Remove columns that we dont needed, rename columns names to make it  clear\n
df_clean=df_games.drop(columns=['localtime', 'date','tz','is_grand_final','complete','is_final','id',
                                'hteamid','ateamid','winnerteamid','updated','round'])
#rename column 
Games=df_clean.rename(columns={'abehinds': 'away_behinds', 'hteam': 'home_team','hgoals': 'home_goals','hscore': 'home_score','hgoals': 'home_goals','ascore': 'away_score','agoals': 'away_goals','ateam': 'away_team','hbehinds': 'home_behinds'}) 
Games.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2220 entries, 0 to 2219
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   away_score    2220 non-null   int64 
 1   venue         2220 non-null   object
 2   away_team     2220 non-null   object
 3   home_goals    2220 non-null   int64 
 4   away_behinds  2220 non-null   int64 
 5   winner        2203 non-null   object
 6   roundname     2220 non-null   object
 7   home_score    2220 non-null   int64 
 8   home_team     2220 non-null   object
 9   away_goals    2220 non-null   int64 
 10  home_behinds  2220 non-null   int64 
 11  year          2220 non-null   int64 
dtypes: int64(7), object(5)
memory usage: 208.2+ KB


In [48]:
#getting a list of venue names 
Games['venue'].unique()

array(['M.C.G.', 'S.C.G.', 'Docklands', 'Carrara', 'Adelaide Oval',
       'Subiaco', 'Sydney Showground', 'Gabba', 'Bellerive Oval',
       'UNSW Canberra Oval', 'University of Tasmania Stadium',
       'Adelaide Arena at Jiangwan Stadium', 'GMHBA Stadium',
       'Traeger Park', 'Marrara Oval', "Cazaly's Stadium", 'Mars Stadium',
       'Optus Stadium', 'Manuka Oval', 'Kardinia Park', 'York Park',
       'Stadium Australia', 'Football Park', 'Blacktown', 'Wellington',
       'Perth Stadium', 'Marvel Stadium', 'Eureka Stadium',
       'Jiangwan Stadium', 'Riverway Stadium'], dtype=object)

In [49]:
Players=pd.read_csv('stats.csv', encoding="ISO-8859-1") 
Players.info()
Players.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89470 entries, 0 to 89469
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gameId                   89470 non-null  object
 1   team                     89470 non-null  object
 2   year                     89470 non-null  int64 
 3   round                    89470 non-null  object
 4   playerId                 89470 non-null  int64 
 5   displayName              89470 non-null  object
 6   gameNumber               89470 non-null  int64 
 7   Disposals                89470 non-null  int64 
 8   Kicks                    89470 non-null  int64 
 9   Marks                    89470 non-null  int64 
 10  Handballs                89470 non-null  int64 
 11  Goals                    89470 non-null  int64 
 12  Behinds                  89470 non-null  int64 
 13  Hit Outs                 89470 non-null  int64 
 14  Tackles                  89470 non-nul

,gameId,team,year,round,playerId,displayName,gameNumber,Disposals,Kicks,Marks,...,Brownlow Votes,Contested Possessions,Uncontested Possessions,Contested Marks,Marks Inside 50,One Percenters,Bounces,Goal Assists,% Played,Subs
0,2021R104,Adelaide,2021,R1,2021661124,"Berry, Sam",1,8,6,1,...,0,4,5,0,0,0,0,1,80,-
1,2021R104,Adelaide,2021,R1,2012662083,"Brown, Luke",168,5,2,0,...,0,2,3,0,0,0,0,0,23,Off
2,2021R104,Adelaide,2021,R1,2020665315,"Butts, Jordon",3,10,5,3,...,0,5,5,1,0,8,0,0,93,-
3,2021R104,Adelaide,2021,R1,2018689604,"Doedee, Tom",31,13,9,4,...,0,8,6,0,0,7,0,0,84,-
4,2021R104,Adelaide,2021,R1,2018703883,"Frampton, Billy",9,14,10,8,...,0,5,9,3,4,2,0,0,90,-


In [50]:
Players=Players[['team','displayName','Disposals','Kicks','Marks','Handballs','Goals','Behinds','Hit Outs','Tackles']]
Players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89470 entries, 0 to 89469
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   team         89470 non-null  object
 1   displayName  89470 non-null  object
 2   Disposals    89470 non-null  int64 
 3   Kicks        89470 non-null  int64 
 4   Marks        89470 non-null  int64 
 5   Handballs    89470 non-null  int64 
 6   Goals        89470 non-null  int64 
 7   Behinds      89470 non-null  int64 
 8   Hit Outs     89470 non-null  int64 
 9   Tackles      89470 non-null  int64 
dtypes: int64(8), object(2)
memory usage: 6.8+ MB


In [51]:
Venue=pd.read_csv('venue_coord.csv')
#engine='python',sep='\t',error_bad_lines=False) 
Venue.info()
#df_coord.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Venue       30 non-null     object 
 1   Latitude    29 non-null     float64
 2   Longtitude  29 non-null     float64
dtypes: float64(2), object(1)
memory usage: 848.0+ bytes


Part II: Connecting to SQL 

In [55]:
# connecting 
connection_string = "postgres:iloveyou@localhost:5432/project 2"
engine = create_engine(f'postgresql://{connection_string}')

In [57]:
# transform data
Games.to_sql(name='Games', con=engine, if_exists='replace', index=False)
Players.to_sql(name='Players', con=engine, if_exists='replace', index=False)
Venue.to_sql(name='Venue', con=engine, if_exists='replace', index=False)

In [60]:
engine.table_names()

<ipython-input-60-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['Games', 'Players', 'Venue']

In [61]:
pd.read_sql_query('select * from "Games"', con=engine).head()

,away_score,venue,away_team,home_goals,away_behinds,winner,roundname,home_score,home_team,away_goals,home_behinds,year
0,132,M.C.G.,Richmond,14,12,Richmond,Round 1,89,Carlton,20,5,2017
1,100,M.C.G.,Western Bulldogs,12,10,Western Bulldogs,Round 1,86,Collingwood,15,14,2017
2,110,S.C.G.,Port Adelaide,12,8,Port Adelaide,Round 1,82,Sydney,17,10,2017
3,120,Docklands,Melbourne,13,12,Melbourne,Round 1,90,St Kilda,18,12,2017
4,98,Carrara,Brisbane Lions,14,8,Brisbane Lions,Round 1,96,Gold Coast,15,12,2017
